In [1]:
%cd "/content/drive/My Drive/Lorenzo/ego-rnn-ss-task"

/content/drive/My Drive/Lorenzo/ego-rnn-ss-task


In [0]:
import numpy as np
from torchvision import transforms
import cv2
from objectAttentionModelConvLSTM import *
from attentionMapModel import attentionMap
from PIL import Image
import os
import imageio

In [6]:
DEVICE = "cuda"

####################Model definition###############################
num_classes = 61  # Classes in the pre-trained model
mem_size = 512
#model_state_dict = '/content/drive/My Drive/FINAL_LOGS/200+150epochs_RGB_16frames/test_2/model_rgb_state_dict.pth'  # Weights of the pre-trained model
#model_state_dict = "/content/drive/My Drive/Lorenzo/ego-rnn-latest/200+150epochs_RGB_16frames/test_2/model_rgb_state_dict.pth"
model_state_dict = "/content/drive/My Drive/Lorenzo/ego-rnn-ss-task/rgb_16frames_regression_kl/model_rgb_state_dict.pth"
model = attentionModel(num_classes=num_classes, mem_size=mem_size)
model.load_state_dict(torch.load(model_state_dict), strict=False)
model_backbone = model.resNet
attentionMapModel = attentionMap(model_backbone).to(DEVICE)
attentionMapModel.train(False)
for params in attentionMapModel.parameters():
    params.requires_grad = False
###################################################################

/content/drive/My Drive/Lorenzo/ego-rnn-ss-task/MyConvLSTMCell.py:31: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_i_xx.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-ss-task/MyConvLSTMCell.py:32: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.constant(self.conv_i_xx.bias, 0)
/content/drive/My Drive/Lorenzo/ego-rnn-ss-task/MyConvLSTMCell.py:33: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_i_hh.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-ss-task/MyConvLSTMCell.py:35: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_f_xx.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-ss-task/MyConvLSTMCell.py:36: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.c

In [7]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

preprocess1 = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
])

preprocess2 = transforms.Compose([
    transforms.ToTensor(),
    normalize])

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [8]:
fl_name_in = './attention_test/frames/'
fl_name_out = './attention_test/open_ketchup/'

# load video and get frames #

'''
cam = cv2.VideoCapture("./attention_test/grab_glass.mp4")            # load video 

currentframe = 0
while (True):
    ret, frame = cam.read()
    if ret:
        name = fl_name_in + "/" + str(currentframe).zfill(3) + '.jpg'
        cv2.imwrite(name, frame)
        currentframe += 1
    else:
        break
'''

frames = []
for i, img_name in enumerate(sorted(os.listdir(fl_name_in))):
    img_pil = Image.open(os.path.join(fl_name_in, img_name))                    # open image
    img_pil1 = preprocess1(img_pil)                                             # apply preprocessing (scaling and cropping)
    img_size = img_pil1.size
    size_upsample = (img_size[0], img_size[1])
    img_tensor = preprocess2(img_pil1)                                          # convert to tensor
    img_variable = Variable(img_tensor.unsqueeze(0).to(DEVICE))                 # send to gpu
    img = np.asarray(img_pil1)                                                  # keep original image (transformed)
    attentionMap_image = attentionMapModel(img_variable, img, size_upsample)    # compute attention map and stack it on original image
    cv2.imwrite(os.path.join(fl_name_out, img_name), attentionMap_image)        # save image

    frames.append(imageio.imread(os.path.join(fl_name_out, img_name)))          # keep image, read with imageio

    print(f"frame_{i}")

kargs = {'duration': 0.05}                                                      # duration of single frame
imageio.mimsave(fl_name_out + "/frames.gif", frames, **kargs)                   # create gif

frame_0
frame_1
frame_2
frame_3
frame_4
frame_5
frame_6
frame_7
frame_8
frame_9
frame_10
frame_11
frame_12
frame_13
frame_14
frame_15
frame_16
frame_17
frame_18
frame_19
frame_20
frame_21
